In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone -b feature/add_rl_part https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 398 (delta 105), reused 5 (delta 3), pack-reused 227 (from 1)
Receiving objects: 100% (398/398), 137.13 KiB | 6.86 MiB/s, done.
Resolving deltas: 100% (224/224), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.9/259.9 kB 11.1 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=118d57f7924bbe91c308a35acedaeb998ca1797d8f38ac71a8e14019e3b9da25
  Stored in directory: /root/.ca

In [2]:
import sys
import pathlib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [5]:
from src.backtest.behavior_policy import run_behavior_policy_with_guards
from src.rl.state_builder import ensure_z_column, build_rl_dataset

In [6]:
# 1) Load final stacked dataset from 03_calibrate_and_stack
final_path = "/kaggle/input/01-prepare-data/final_dataset.csv"
df = pd.read_csv(final_path, parse_dates=["Date"])

In [7]:
# 2) Ensure we have a z column (dummy for now)
z_col ="dummy_z"
df = ensure_z_column(df, z_col=z_col, seed=42)

Using randomly generated dummy z column


In [8]:
# 3) Run dumb behavior policy + guards
df_policy = run_behavior_policy_with_guards(
    df,
    z_col="dummy_z",    # later change to "z" when you have real alpha
    nav=1_000_000.0,
    allow_short=False,
    trading_enabled=True,
)

In [9]:
# 4) Build RL training dataset (s, a, r, s')
extra_state_cols = ["has_news", "disagreement"]
rl_df = build_rl_dataset(
    df_policy,
    z_col="dummy_z",
    reward_return_col="excess_return",
    action_col="action_weight_raw",
    weight_col="weight_after_guards",
    extra_state_cols=None,  # only if those exist
    cost_bps=10.0,
    lambda_risk=0.1,
)
rl_df

,Date,ticker,reward,action,done,state_dummy_z,state_VIX_z,state_spread_z,state_weight_after_guards,next_state_dummy_z,next_state_VIX_z,next_state_spread_z,next_state_weight_after_guards
0,2010-01-04 00:00:00+00:00,AAPL,-0.000000,0.00,0,0.304717,-1.249591,-0.231379,0.000000,-1.039984,-1.314181,-0.015455,0.000000
8,2010-01-05 00:00:00+00:00,AAPL,-0.000000,0.00,0,-1.039984,-1.314181,-0.015455,0.000000,0.750451,-1.323599,0.871336,0.000000
16,2010-01-06 00:00:00+00:00,AAPL,-0.000000,0.00,0,0.750451,-1.323599,0.871336,0.000000,0.940565,-1.324017,1.082035,0.006999
24,2010-01-07 00:00:00+00:00,AAPL,0.000002,0.01,0,0.940565,-1.324017,1.082035,0.006999,-1.951035,-1.413993,0.266584,0.000000
32,2010-01-08 00:00:00+00:00,AAPL,-0.000007,0.00,0,-1.951035,-1.413993,0.266584,0.000000,-1.302180,-1.465035,0.106547,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21858,2018-12-20 00:00:00+00:00,WMT,0.000000,0.00,0,-2.315280,2.598611,3.000000,0.000000,-1.540832,2.909232,-0.975390,0.000000
21868,2018-12-21 00:00:00+00:00,WMT,0.000000,0.00,0,-1.540832,2.909232,-0.975390,0.000000,1.189034,3.000000,2.115415,0.000000
21878,2018-12-24 00:00:00+00:00,WMT,0.000000,0.02,0,1.189034,3.000000,2.115415,0.000000,0.369053,2.812017,-2.678646,0.003489
21888,2018-12-26 00:00:00+00:00,WMT,-0.000012,0.01,0,0.369053,2.812017,-2.678646,0.003489,-0.353816,2.675121,-2.617727,0.000000


In [10]:
!rm -rf /kaggle/working/multimodal-eq-sizing